# Box Office 7 Domestic Weekly Revenue Prediction Algorithm by Faiyaz Ahmed.
Codebook 1: Pre-processing
This project focuses on building a data-driven algorithm to predict domestic box office revenue over the first seven days of release. Accurate early-week revenue forecasting is critical for studios, distributors, and exhibitors, as it directly informs marketing spend allocation, screen count decisions, and risk assessment before and immediately after release.

The objective of this codebook is to establish a clean, reproducible preprocessing pipeline that transforms raw, heterogeneous movie data into a model-ready dataset. The preprocessing stage consolidates information from multiple sources and standardizes key attributes such as release dates, countries, languages, budgets, marketing signals, and calendar-based effects (weekends, holidays, and competitive releases). Special attention is given to handling missing values, encoding categorical variables, normalizing skewed financial features, and preventing data leakage.

This notebook represents Codebook 1: Pre-processing, which serves as the foundation for all downstream modeling work. The outputs of this stage are structured train/validation/test splits and engineered features suitable for baseline statistical models (e.g., log-linear regression) as well as more advanced machine learning approaches (e.g., gradient boosting models). Ensuring consistency, transparency, and scalability at this stage is essential for reliable performance evaluation and meaningful interpretation of model results in later stages of the project.

# All Imports

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt, timedelta,date
import re
!pip install country_converter
import country_converter as coco

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.5/47.5 kB 1.5 MB/s eta 0:00:00


#Loading dataset

In [2]:
box=pd.read_csv('/content/drive/MyDrive/Box office prediction dataset/master_reduced_2.csv')

In [3]:
box.columns

Index(['Unnamed: 0', 'Release', 'Seven_Day_Total', 'Theaters', 'Distributor',
       'normalized_title', 'year', 'tmdb_id', 'original_language', 'runtime',
       'release_date', 'budget', 'revenue', 'spoken_languages', 'genres',
       'production_companies', 'production_countries', 'collection_id',
       'collection_name', 'collection_total_parts', 'collection_total_revenue',
       'cast_top5_names', 'cast_top5_popularity_sum', 'directors', 'writers',
       'producers', 'keywords', 'keywords_count', 'certification',
       'num_trailers', 'num_teasers', 'num_featurettes', 'num_clips',
       'num_posters', 'num_backdrops', 'imdb_id', 'total_marketing_assets',
       'log_total_marketing_assets', 'director_weighted_rating'],
      dtype='object')

In [4]:
box.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2573 entries, 0 to 2572
Data columns (total 39 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  2573 non-null   int64  
 1   Release                     2573 non-null   object 
 2   Seven_Day_Total             2573 non-null   float64
 3   Theaters                    2573 non-null   object 
 4   Distributor                 2566 non-null   object 
 5   normalized_title            2573 non-null   object 
 6   year                        2573 non-null   int64  
 7   tmdb_id                     2573 non-null   float64
 8   original_language           2573 non-null   object 
 9   runtime                     2573 non-null   float64
 10  release_date                2573 non-null   object 
 11  budget                      2573 non-null   float64
 12  revenue                     2573 non-null   float64
 13  spoken_languages            2567 

##**Dataset Description**

The dataset consists of **2,573 theatrically released films**, each represented by **39 variables** combining box office outcomes, movie metadata, talent-related attributes, and marketing signals. The primary target variable is **Seven_Day_Total**, which captures domestic box office revenue during the first week of release. The dataset integrates information from multiple sources, including title and release metadata, distributor and theater information, TMDB-linked identifiers, cast and crew details, franchise indicators, and marketing asset counts (trailers, posters, clips, and backdrops). It includes both raw and engineered features—such as log-transformed marketing intensity and weighted director ratings—allowing the model to capture non-linear effects and historical performance signals. Overall, the dataset provides a balanced mix of financial, creative, temporal, and promotional variables, making it well-suited for early-stage box office revenue prediction.


#Feature Engineering

##Creating a marketing/hype metric.
This step creates a consolidated **marketing/hype metric** by aggregating all available promotional assets—trailers, teasers, featurettes, clips, posters, and backdrops—into a single measure of marketing intensity. Individually, these variables are highly correlated and collectively represent the overall visibility and promotional push behind a film prior to release. Summing them reduces feature redundancy while preserving the total marketing signal. The subsequent **log transformation** is applied to address strong right-skewness in promotional counts, stabilize variance, and better capture diminishing returns, where additional marketing assets contribute progressively smaller marginal effects on opening-week revenue.


In [5]:
box["total_marketing_assets"] = (
      box["num_trailers"]
    + box["num_teasers"]
    + box["num_featurettes"]
    + box["num_clips"]
    + box["num_posters"]
    + box["num_backdrops"]
)

box["log_total_marketing_assets"] = np.log1p(box["total_marketing_assets"])

In [6]:
box.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2573 entries, 0 to 2572
Data columns (total 39 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  2573 non-null   int64  
 1   Release                     2573 non-null   object 
 2   Seven_Day_Total             2573 non-null   float64
 3   Theaters                    2573 non-null   object 
 4   Distributor                 2566 non-null   object 
 5   normalized_title            2573 non-null   object 
 6   year                        2573 non-null   int64  
 7   tmdb_id                     2573 non-null   float64
 8   original_language           2573 non-null   object 
 9   runtime                     2573 non-null   float64
 10  release_date                2573 non-null   object 
 11  budget                      2573 non-null   float64
 12  revenue                     2573 non-null   float64
 13  spoken_languages            2567 

##Creating Seasonality indicators

Coding time variables into proper date-time format

In [7]:
weekend_days={4,5,6}
box['release_date']=pd.to_datetime(box['release_date'],errors='coerce')
box['day_of_the_week']=box['release_date'].dt.weekday
box['weekend_release']=box['day_of_the_week'].isin(weekend_days).astype(int)

###US box office holiday calendar construction

Defines helper functions to compute date-based U.S. holidays that are known to influence box office performance. It programmatically calculates holidays that fall on variable weekdays (e.g., MLK Day, Presidents’ Day, Memorial Day, Labor Day, Thanksgiving) as well as fixed-date holidays (New Year’s Day, Independence Day, Christmas). By generating these dates dynamically for any year, the model avoids hard-coded calendars and remains robust across different release years.

In [8]:
# ---------- helpers ----------
def nth_weekday(year, month, weekday, n):
    """weekday: 0=Mon ... 6=Sun; n=1..5"""
    d = date(year, month, 1)
    shift = (weekday - d.weekday()) % 7
    d = d + timedelta(days=shift)  # first target weekday in month
    return d + timedelta(weeks=n-1)

def last_weekday(year, month, weekday):
    d = date(year, month + 1, 1) - timedelta(days=1) if month < 12 else date(year, 12, 31)
    while d.weekday() != weekday:
        d -= timedelta(days=1)
    return d

def us_boxoffice_holidays(year):
    """Return dict{name: date} for major US box-office holidays."""
    return {
        "new_year":       date(year, 1, 1),
        "mlk_day":        nth_weekday(year, 1, 0, 3),                 # 3rd Mon Jan
        "presidents_day": nth_weekday(year, 2, 0, 3),                 # 3rd Mon Feb
        "memorial_day":   last_weekday(year, 5, 0),                   # last Mon May
        "independence":   date(year, 7, 4),                           # Jul 4
        "labor_day":      nth_weekday(year, 9, 0, 1),                 # 1st Mon Sep
        "thanksgiving":   nth_weekday(year, 11, 3, 4),                # 4th Thu Nov
        "christmas":      date(year, 12, 25),                         # Dec 25
    }

###Holiday influence windows

Specifies pre-holiday influence windows for each major box office holiday, measured in days leading up to the holiday itself. These windows capture real-world release strategies where films open shortly before a holiday to benefit from extended weekends and higher audience turnout (notably for Christmas and Thanksgiving). Defining these windows allows the model to account for anticipatory demand rather than only the holiday date itself.

In [9]:
# Per-holiday windows (days). Widen Christmas for box-office seasonality.
HOLIDAY_WINDOWS = {
    "new_year": 3,
    "mlk_day": 2,
    "presidents_day": 2,
    "memorial_day": 3,
    "independence": 2,
    "labor_day": 2,
    "thanksgiving": 3,
    "christmas": 7,  # captures pre-Christmas launches (e.g., Dec 18–25)
}

def is_us_holiday(d):
    """Return (is_holiday:int, holiday_type:str) using US box-office windows."""
    if pd.isna(d):
        return 0, "none"
    d = pd.to_datetime(d).date()
    hols = us_boxoffice_holidays(d.year)
    best_name, best_gap = "none", 10**9
    for name, anchor in hols.items():
        gap = abs((d - anchor).days)
        if gap <= HOLIDAY_WINDOWS[name] and gap < best_gap:
            best_name, best_gap = name, gap
    return (1, best_name) if best_name != "none" else (0, "none")

###Holiday flag and labeling logic

Applying the holiday logic to the dataset by checking whether each film’s release date falls within any defined holiday window. It outputs both a binary indicator (is_holiday) and a categorical label (holiday_type) identifying the nearest relevant holiday. This converts complex calendar effects into clean, interpretable features that can be used directly in regression and tree-based models without introducing temporal leakage.

In [99]:
box["release_date"] = pd.to_datetime(box["release_date"], errors="coerce")
flags = box["release_date"].apply(is_us_holiday)
box["is_holiday"], box["holiday_type"] = zip(*flags)
box["is_holiday"] =box["is_holiday"].astype(int)

###Month Sin and Cos
This block creates a **seasonality feature** from `release_date`. It first checks that all dates are parseable and non-null. From each date it extracts `release_month` (1–12), then encodes the month on a **cyclical circle** using `sin`/`cos`:
`angle = 2π * (month−1) / 12`, `month_sin = sin(angle)`, `month_cos = cos(angle)`.
Cyclical encoding preserves the fact that December and January are adjacent and prevents models from interpreting months as an ordinal scale. The resulting features are bounded in [−1, 1], stable across years, and compact (`float32`), making them suitable for linear and tree models to learn recurring box-office patterns (e.g., summer/holiday peaks) without leakage.


In [26]:
DATE_COL='release_date'

In [27]:
n_bad = box[DATE_COL].isna().sum()
print(f"Unparseable dates: {n_bad}")

Unparseable dates: 0


In [28]:
box[DATE_COL].isnull().sum()

np.int64(0)

In [29]:
box["release_month"] = box[DATE_COL].dt.month.astype(int)   # 1..12

In [30]:
angle = 2 * np.pi * (box["release_month"].astype(float) - 1) / 12.0
box["month_sin"] = np.sin(angle).astype("float32")
box["month_cos"] = np.cos(angle).astype("float32")

In [31]:
print(box[[DATE_COL, "year", "release_month", "month_sin", "month_cos"]].head())

  release_date  year  release_month  month_sin     month_cos
0   2021-12-23  2021             12  -0.500000  8.660254e-01
1   2016-03-10  2016              3   0.866025  5.000000e-01
2   2000-10-07  2000             10  -1.000000 -1.836970e-16
3   2018-01-18  2018              1   0.000000  1.000000e+00
4   2013-10-18  2013             10  -1.000000 -1.836970e-16


##Encoding Languages into ISO Language Codes
This step standardizes the **spoken language information** by converting language names into **ISO language codes**, ensuring consistency across the dataset. Raw language fields often contain variations in spelling, formatting, or multiple languages per film, which can introduce noise and inflate the number of unique categories. Encoding languages into ISO codes provides a normalized, machine-readable representation that is compact, unambiguous, and easier to aggregate or multi-label encode. This normalization is especially important for modeling, as it enables consistent handling of multilingual films, reduces sparsity in categorical features, and improves generalization when training both linear and tree-based models.


In [13]:
box['spoken_languages'].unique()

array(['hi', 'en', 'English', 'English; French; German',
       'Spanish; English', 'Hindi', 'hi, ta', 'fr', 'en, sv',
       'en, de, es', 'English; Spanish', 'en, ja', 'en, ka, fr', 'fr, en',
       'en, ko', 'English; Portuguese; Spanish', 'en, it, pt', 'te',
       'en, es', 'en, de', 'English; Russian',
       'English; French; Italian; Latin; Spanish',
       'English; German; Italian', 'en, ar',
       'Spanish; English; Portuguese', 'Polish; English',
       'English; Russian; Spanish', 'Spanish', 'Telugu', 'en, ru',
       'English; Norwegian', 'Italian; English; French', 'en, pt',
       'German; English', 'English; French', 'en, es, th',
       'ar, en, fr, de', 'Russian; English', 'Korean',
       'Spanish; English; Cantonese',
       'English; Arabic; Spanish; Italian; Persian; Polish; Turkish',
       'es, fr, en', 'en, hi', 'en, cn, zh, ja, th', nan, 'en, fr, ru',
       'en, ru, hy', 'zh, en, ko', 'Mandarin; English; Korean', 'en, he',
       'ar, en, de, fa', 'Mandarin

In [14]:

# 1) Canonical language mapping (extend as needed)
LANG_MAP = {
    # core
    "en":"en","english":"en",
    "hi":"hi","hindi":"hi",
    "bn":"bn","bangla":"bn","bengali":"bn",
    "ur":"ur","urdu":"ur",
    "fr":"fr","french":"fr",
    "es":"es","spanish":"es",
    "de":"de","german":"de",
    "it":"it","italian":"it",
    "pt":"pt","portuguese":"pt","português":"pt","portugese":"pt",
    "ru":"ru","russian":"ru",
    "zh":"zh","chinese":"zh","mandarin":"zh","cantonese":"zh",  # cast_poplapse to zh; split later if you track yue/zh-cn
    "ja":"ja","japanese":"ja",
    "ko":"ko","korean":"ko",
    "ar":"ar","arabic":"ar",
    "tr":"tr","turkish":"tr",
    "th":"th","thai":"th",
    "nl":"nl","dutch":"nl",
    "sv":"sv","swedish":"sv",
    "no":"no","norwegian":"no","bokmål":"no","nynorsk":"no",
    "da":"da","danish":"da",
    "fi":"fi","finnish":"fi",
    "pl":"pl","polish":"pl",
    "el":"el","greek":"el",
    "he":"he","hebrew":"he",
    "fa":"fa","persian":"fa","farsi":"fa",
    "cs":"cs","czech":"cs",
    "sk":"sk","slovak":"sk",
    "uk":"uk","ukrainian":"uk",
    "ro":"ro","romanian":"ro",
    "hu":"hu","hungarian":"hu",
    "sr":"sr","serbian":"sr",
    "hr":"hr","croatian":"hr",
    "bg":"bg","bulgarian":"bg",
    "et":"et","estonian":"et",
    "lv":"lv","latvian":"lv",
    "lt":"lt","lithuanian":"lt",
    "sl":"sl","slovenian":"sl",
    "id":"id","indonesian":"id",
    "ms":"ms","malay":"ms",
    "vi":"vi","vietnamese":"vi",
    "ta":"ta","tamil":"ta",
    "te":"te","telugu":"te",
    "mr":"mr","marathi":"mr",
    "pa":"pa","punjabi":"pa","panjabi":"pa",
    "gu":"gu","gujarati":"gu",
    "kn":"kn","kannada":"kn",
    "ml":"ml","malayalam":"ml",
    "si":"si","sinhala":"si","sinhalese":"si",
    # add any leftovers you see in your uniques
}

# 2) Parser/standardizer
DELIMS = re.compile(r"[;,/|]|(?<![a-z])\s+\&\s+|,|\s{2,}")

def normalize_lang_cell(x):
    if pd.isna(x) or str(x).strip()=="":
        return []
    s = str(x).lower()
    # replace ' and ' with delimiter to catch "english and french"
    s = re.sub(r"\band\b", ",", s)
    # split on common delimiters
    parts = [p.strip() for p in re.split(r"[,\;/\|]", s) if p.strip()!=""]
    canon = []
    for p in parts:
        # remove non-letters for tokens like 'en,' 'fr)' etc.
        p_clean = re.sub(r"[^a-z]", "", p)
        code = LANG_MAP.get(p_clean)
        if code:
            canon.append(code)
    # dedupe + sort for stable representation
    return sorted(set(canon))

# 3) Apply to your dataframe cast_popumn
# assumes your cast_popumn is named 'spoken_languages'
box["spoken_languages_iso"] = box["spoken_languages"].apply(normalize_lang_cell)

In [15]:
box['original_language_iso']=box['original_language'].apply(normalize_lang_cell)

In [16]:
box['spoken_languages_iso'] = box['spoken_languages_iso'].apply(
    lambda x: tuple(sorted(x)) if isinstance(x, list) else x
)

box['original_language_iso'] = box['original_language_iso'].apply(
    lambda x: tuple(sorted(x)) if isinstance(x, list) else x
)

box.drop_duplicates(inplace=True)


In [17]:
box.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2573 entries, 0 to 2572
Data columns (total 45 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Unnamed: 0                  2573 non-null   int64         
 1   Release                     2573 non-null   object        
 2   Seven_Day_Total             2573 non-null   float64       
 3   Theaters                    2573 non-null   object        
 4   Distributor                 2566 non-null   object        
 5   normalized_title            2573 non-null   object        
 6   year                        2573 non-null   int64         
 7   tmdb_id                     2573 non-null   float64       
 8   original_language           2573 non-null   object        
 9   runtime                     2573 non-null   float64       
 10  release_date                2573 non-null   datetime64[ns]
 11  budget                      2573 non-null   float64     

In [18]:
box['spoken_languages_iso'] = box['spoken_languages_iso'].apply(
    lambda x: list(x) if isinstance(x, tuple) else x
)
box['original_language_iso'] = box['original_language_iso'].apply(
    lambda x: list(x) if isinstance(x, tuple) else x
)


##Creating competition flags
This block engineers a same-day competition signal, because movies opening alongside others typically split audience attention and can depress first-week revenue. It first counts how many titles share a release date, then subtracts one to remove the current film, yielding the number of same-day competitors. A compact binary flag (has_same_day_competitor) captures the presence/absence of competition for models that benefit from simple indicators, while the integer count preserves intensity for tree/GBM models. fillna(0) ensures robust handling when counts are missing, and integer dtypes keep memory/IO efficient.

In [19]:
# 1) same-day count (including self)
box["same_release_date_count"] = box.groupby("release_date")["imdb_id"].transform("count").astype("Int64")

# 3) competitors on the same day (exclude self)
box["same_day_competitors"] = (box["same_release_date_count"] - 1).clip(lower=0).astype("Int64")

# 4) binary flag
box["has_same_day_competitor"] = (box["same_day_competitors"] > 0).astype("int8")


In [20]:
box["same_day_competitors"] = box["same_day_competitors"].fillna(0).astype(int)


In [21]:
box.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2573 entries, 0 to 2572
Data columns (total 48 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Unnamed: 0                  2573 non-null   int64         
 1   Release                     2573 non-null   object        
 2   Seven_Day_Total             2573 non-null   float64       
 3   Theaters                    2573 non-null   object        
 4   Distributor                 2566 non-null   object        
 5   normalized_title            2573 non-null   object        
 6   year                        2573 non-null   int64         
 7   tmdb_id                     2573 non-null   float64       
 8   original_language           2573 non-null   object        
 9   runtime                     2573 non-null   float64       
 10  release_date                2573 non-null   datetime64[ns]
 11  budget                      2573 non-null   float64     

In [22]:
box.head()

,Unnamed: 0,Release,Seven_Day_Total,Theaters,Distributor,normalized_title,year,tmdb_id,original_language,runtime,...,director_weighted_rating,day_of_the_week,weekend_release,is_holiday,holiday_type,spoken_languages_iso,original_language_iso,same_release_date_count,same_day_competitors,has_same_day_competitor
0,2,'83,2595812.0,486,Reliance Entertainment,83,2021,601337.0,hi,160.0,...,5.978574,3,0,1,christmas,[hi],[hi],1,0,0
1,4,10 Cloverfield Lane,32676639.0,"3,391",Paramount Pictures,10 cloverfield lane,2016,333371.0,en,104.0,...,7.345529,3,0,0,none,[en],[en],2,1,1
2,9,102 Dalmatians,27376052.0,"2,704",Walt Disney Studios Motion Pictures,102 dalmatians,2000,10481.0,en,100.0,...,6.980777,5,1,0,none,[en],[en],1,0,0
3,13,12 Strong,21124610.0,"3,002",Warner Bros.,12 strong,2018,429351.0,en,130.0,...,6.345000,3,0,0,none,[en],[en],2,1,1
4,14,12 Years a Slave,1259943.0,19,Fox Searchlight Pictures,12 years a slave,2013,76203.0,en,134.0,...,5.996000,4,1,0,none,[en],[en],3,2,1


Saving checkpoint, not relevant to model

In [23]:
box.to_csv('/content/drive/MyDrive/Box office prediction dataset/box_final.csv',index=False)

In [24]:
box.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2573 entries, 0 to 2572
Data columns (total 48 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Unnamed: 0                  2573 non-null   int64         
 1   Release                     2573 non-null   object        
 2   Seven_Day_Total             2573 non-null   float64       
 3   Theaters                    2573 non-null   object        
 4   Distributor                 2566 non-null   object        
 5   normalized_title            2573 non-null   object        
 6   year                        2573 non-null   int64         
 7   tmdb_id                     2573 non-null   float64       
 8   original_language           2573 non-null   object        
 9   runtime                     2573 non-null   float64       
 10  release_date                2573 non-null   datetime64[ns]
 11  budget                      2573 non-null   float64     

In [25]:
box.head()

,Unnamed: 0,Release,Seven_Day_Total,Theaters,Distributor,normalized_title,year,tmdb_id,original_language,runtime,...,director_weighted_rating,day_of_the_week,weekend_release,is_holiday,holiday_type,spoken_languages_iso,original_language_iso,same_release_date_count,same_day_competitors,has_same_day_competitor
0,2,'83,2595812.0,486,Reliance Entertainment,83,2021,601337.0,hi,160.0,...,5.978574,3,0,1,christmas,[hi],[hi],1,0,0
1,4,10 Cloverfield Lane,32676639.0,"3,391",Paramount Pictures,10 cloverfield lane,2016,333371.0,en,104.0,...,7.345529,3,0,0,none,[en],[en],2,1,1
2,9,102 Dalmatians,27376052.0,"2,704",Walt Disney Studios Motion Pictures,102 dalmatians,2000,10481.0,en,100.0,...,6.980777,5,1,0,none,[en],[en],1,0,0
3,13,12 Strong,21124610.0,"3,002",Warner Bros.,12 strong,2018,429351.0,en,130.0,...,6.345000,3,0,0,none,[en],[en],2,1,1
4,14,12 Years a Slave,1259943.0,19,Fox Searchlight Pictures,12 years a slave,2013,76203.0,en,134.0,...,5.996000,4,1,0,none,[en],[en],3,2,1


##Log Transformations

A log version of **budget** stabilizes the feature and makes its relationship to revenue easier for models to learn. Movie budgets are extremely right-skewed (a few very large values), which can dominate loss functions and create heteroscedastic residuals. Taking `log1p(budget)`:

* compresses extreme values and reduces skew,
* models **diminishing returns** (each extra dollar has smaller marginal impact),
* makes linear coefficients interpretable in % terms (≈ elasticity),
* is robust to zeros via `log1p`,
* often improves both linear baselines and tree/GBM performance.


In [34]:
budget='budget'

In [35]:
# Sanity checks
assert (box[budget] >= 0).all(), "Budget contains negative values"

In [36]:
# Log transform
box["budget_log"] = np.log1p(box[budget].astype(float))

In [37]:
print(box[[budget, "budget_log"]].head())

       budget  budget_log
0  36786988.0   17.420655
1  15000000.0   16.523561
2  85000000.0   18.258162
3  35000000.0   17.370859
4  20000000.0   16.811243


In [38]:
box.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2573 entries, 0 to 2572
Data columns (total 52 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Unnamed: 0                  2573 non-null   int64         
 1   Release                     2573 non-null   object        
 2   Seven_Day_Total             2573 non-null   float64       
 3   Theaters                    2573 non-null   object        
 4   Distributor                 2566 non-null   object        
 5   normalized_title            2573 non-null   object        
 6   year                        2573 non-null   int64         
 7   tmdb_id                     2573 non-null   float64       
 8   original_language           2573 non-null   object        
 9   runtime                     2573 non-null   float64       
 10  release_date                2573 non-null   datetime64[ns]
 11  budget                      2573 non-null   float64     

The **cast_top5_popularity_sum** was log-transformed to make the feature more stable and predictive. Popularity scores are typically **right-skewed**, where a small number of films with extremely famous casts dominate the scale. Applying `log1p` compresses these extreme values, reducing skew and variance while preserving relative differences.

This transformation also reflects **diminishing returns**: adding another highly popular actor increases audience pull, but by a smaller marginal amount than moving from an unknown to a moderately known cast. Log transformation improves linearity with box office revenue, prevents a few blockbuster casts from overpowering the model, and helps both linear and tree-based models learn smoother, more generalizable relationships.


In [39]:
 cast_pop = "cast_top5_popularity_sum"   # change if needed

# 1) Sanity checks
assert cast_pop in box.columns, f"{cast_pop} not found in dataframe"
assert (box[cast_pop] >= 0).all(), "Negative values found — log transform not valid"

# 2) Log transform
box[f"{cast_pop}_log"] = np.log1p(box[cast_pop].astype(float))

# 3) Quick verification
print(box[[cast_pop, f"{cast_pop}_log"]].describe())


       cast_top5_popularity_sum  cast_top5_popularity_sum_log
count               2573.000000                   2573.000000
mean                  13.622537                      2.527431
std                    8.523443                      0.576444
min                    0.000000                      0.000000
25%                    8.014700                      2.198857
50%                   11.952300                      2.561273
75%                   17.051600                      2.893234
max                   68.519900                      4.241613


In [40]:
box=box.drop(columns='revenue')

In [42]:
box.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2573 entries, 0 to 2572
Data columns (total 52 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Unnamed: 0                    2573 non-null   int64         
 1   Release                       2573 non-null   object        
 2   Seven_Day_Total               2573 non-null   float64       
 3   Theaters                      2573 non-null   object        
 4   Distributor                   2566 non-null   object        
 5   normalized_title              2573 non-null   object        
 6   year                          2573 non-null   int64         
 7   tmdb_id                       2573 non-null   float64       
 8   original_language             2573 non-null   object        
 9   runtime                       2573 non-null   float64       
 10  release_date                  2573 non-null   datetime64[ns]
 11  budget                        

##Handling Franchises


This block engineers franchise-aware features to capture brand equity and sequel effects, which are strong drivers of opening-week revenue. It first flags whether a title belongs to a collection (`is_franchise`), measures the franchise’s breadth (`franchise_size`) and its diminishing-returns scale (`franchise_size_log`), then orders films within each collection by release date to assign an `installment_number` (1 = first entry, 2 = sequel, etc.) and derives a compact sequel indicator (`is_sequel`). The stable sort and nullable-safe handling ensure deterministic, leak-free labels even when dates or IDs are missing. These features let models distinguish debuts from later entries and quantify how established series size and installment position affect demand.

For predicting a movie like Avatar: Fire and Ash, this was very important

In [43]:
box["is_franchise"] = box["collection_id"].notna().astype("int8")


In [44]:
box["franchise_size"] = (box["collection_total_parts"].fillna(0).astype("int16"))


In [45]:
box["franchise_size_log"] = np.log1p(box["franchise_size"])


In [46]:
# --- settings ---
year_col = "release_date"        # datetime-parsable
CID_COL  = "collection_id"       # franchise/collection id

# 0) Ensure datetime and a clean franchise flag
box["is_franchise"] = box[CID_COL].notna().astype("int8")

# 1) For non-franchise or missing dates → not a sequel
box["installment_number"] = 0  # default for all rows initially

# 2) Within each collection, sort by release date and assign installment numbers
#    Earliest release in a collection → 1; second → 2; etc.
mask = box["is_franchise"].eq(1) & box[year_col].notna()
box_loc = box.loc[mask].copy()

# Add a temporary column for stable sorting based on original index order
box_loc['temp_original_index'] = box_loc.index

# stable sort by date (ties broken by index to keep order deterministic)
box_loc = box_loc.sort_values([CID_COL, year_col, 'temp_original_index'])

# dense rank per collection based on release date, and assign to box_loc
box_loc["installment_number"] = (box_loc.groupby(CID_COL)[year_col].rank(method="first")  # 1,2,3,... within that collection
                                 .astype(int))

# write back the calculated installment numbers to the original 'box' DataFrame
box.loc[box_loc.index, "installment_number"] = box_loc["installment_number"].astype("Int16")

# 3) True sequel flag: installments after the first are sequels
box["is_sequel"] = (box["installment_number"] > 1).astype("int8")

# quick check
print(box[[CID_COL, year_col, "installment_number", "is_sequel"]]
      .sort_values([CID_COL, "installment_number"])
      .head(20))

      collection_id release_date  installment_number  is_sequel
919            84.0   2023-06-25                   1          0
1965          263.0   2012-07-17                   1          0
1476          295.0   2011-05-15                   1          0
1475          295.0   2017-05-23                   2          1
1011          328.0   2015-06-06                   1          0
1013          328.0   2018-06-06                   2          1
1012          328.0   2022-06-01                   3          1
1014          328.0   2025-07-01                   4          1
1502          399.0   2010-07-07                   1          0
2219          399.0   2018-09-05                   2          1
1501          399.0   2025-11-05                   3          1
1939          420.0   2010-12-02                   1          0
1840          528.0   2015-06-23                   1          0
1841          528.0   2019-10-23                   2          1
1711          645.0   2012-10-24        

In [47]:
box[box['is_sequel'] == 1]

,Unnamed: 0,Release,Seven_Day_Total,Theaters,Distributor,normalized_title,year,tmdb_id,original_language,runtime,...,release_month,month_sin,month_cos,budget_log,cast_top5_popularity_sum_log,is_franchise,franchise_size,franchise_size_log,installment_number,is_sequel
18,48,22 Jump Street,71251279.0,"3,306",Sony Pictures Releasing,22 jump street,2014,187017.0,en,112.0,...,6,5.000000e-01,-8.660254e-01,17.727534,2.774387,1,2,1.098612,2,1
27,77,47 Meters Down: Uncaged,11415947.0,"2,853",Entertainment Studios Motion Pictures,47 meters down uncaged,2019,480105.0,en,90.0,...,8,-5.000000e-01,-8.660254e-01,16.300417,1.985131,1,3,1.386294,2,1
39,99,A Bad Moms Christmas,24851318.0,"3,615",STX Entertainment,a bad moms christmas,2017,431530.0,en,104.0,...,8,-5.000000e-01,-8.660254e-01,17.147715,3.184450,1,2,1.098612,2,1
51,145,A Haunted House 2,10980656.0,"2,310",Open Road Films (II),a haunted house 2,2014,184345.0,en,87.0,...,4,1.000000e+00,6.123234e-17,15.201805,2.695579,1,2,1.098612,2,1
52,146,A Haunting in Venice,19052729.0,"3,305",20th Century Studios,a haunting in venice,2023,945729.0,en,104.0,...,9,-8.660254e-01,-5.000000e-01,17.909855,2.422056,1,3,1.386294,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473,7034,War 2,3087065.0,875,Yash Raj Films USA Inc.,war 2,2025,1109086.0,hi,173.0,...,8,-5.000000e-01,-8.660254e-01,17.665658,2.094933,1,9,2.302585,5,1
2477,7040,War for the Planet of the Apes,77350914.0,"4,022",Twentieth Century Fox,war for the planet of the apes,2017,281338.0,en,140.0,...,7,1.224647e-16,-1.000000e+00,18.826146,2.533800,1,4,1.609438,3,1
2537,7225,Wonder Woman 1984,20180074.0,"2,151",Warner Bros.,wonder woman 1984,2020,464052.0,en,151.0,...,12,-5.000000e-01,8.660254e-01,19.113828,2.990886,1,2,1.098612,2,1
2545,7241,Wrath of the Titans,43882091.0,"3,545",Warner Bros.,wrath of the titans,2012,57165.0,en,99.0,...,3,8.660254e-01,5.000000e-01,18.826146,3.259615,1,2,1.098612,3,1


In [48]:
box.head()

,Unnamed: 0,Release,Seven_Day_Total,Theaters,Distributor,normalized_title,year,tmdb_id,original_language,runtime,...,release_month,month_sin,month_cos,budget_log,cast_top5_popularity_sum_log,is_franchise,franchise_size,franchise_size_log,installment_number,is_sequel
0,2,'83,2595812.0,486,Reliance Entertainment,83,2021,601337.0,hi,160.0,...,12,-0.500000,8.660254e-01,17.420655,1.418108,0,0,0.000000,0,0
1,4,10 Cloverfield Lane,32676639.0,"3,391",Paramount Pictures,10 cloverfield lane,2016,333371.0,en,104.0,...,3,0.866025,5.000000e-01,16.523561,2.251302,0,0,0.000000,0,0
2,9,102 Dalmatians,27376052.0,"2,704",Walt Disney Studios Motion Pictures,102 dalmatians,2000,10481.0,en,100.0,...,10,-1.000000,-1.836970e-16,18.258162,2.237929,0,2,1.098612,0,0
3,13,12 Strong,21124610.0,"3,002",Warner Bros.,12 strong,2018,429351.0,en,130.0,...,1,0.000000,1.000000e+00,17.370859,2.940177,0,0,0.000000,0,0
4,14,12 Years a Slave,1259943.0,19,Fox Searchlight Pictures,12 years a slave,2013,76203.0,en,134.0,...,10,-1.000000,-1.836970e-16,16.811243,2.690897,0,0,0.000000,0,0


In [49]:
box.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2573 entries, 0 to 2572
Data columns (total 57 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Unnamed: 0                    2573 non-null   int64         
 1   Release                       2573 non-null   object        
 2   Seven_Day_Total               2573 non-null   float64       
 3   Theaters                      2573 non-null   object        
 4   Distributor                   2566 non-null   object        
 5   normalized_title              2573 non-null   object        
 6   year                          2573 non-null   int64         
 7   tmdb_id                       2573 non-null   float64       
 8   original_language             2573 non-null   object        
 9   runtime                       2573 non-null   float64       
 10  release_date                  2573 non-null   datetime64[ns]
 11  budget                        

Saving checkpoint. Irrelevant

In [50]:
box.to_csv('/content/drive/MyDrive/Box office prediction dataset/master_vol_4.csv')

In [51]:
box=box.drop(columns=['original_language','spoken_languages'])#not needed

Dropping unnecessary columns

In [52]:
box_three=box.drop(columns="imdb_id")

In [53]:
box_three.head()

,Unnamed: 0,Release,Seven_Day_Total,Theaters,Distributor,normalized_title,year,tmdb_id,runtime,release_date,...,release_month,month_sin,month_cos,budget_log,cast_top5_popularity_sum_log,is_franchise,franchise_size,franchise_size_log,installment_number,is_sequel
0,2,'83,2595812.0,486,Reliance Entertainment,83,2021,601337.0,160.0,2021-12-23,...,12,-0.500000,8.660254e-01,17.420655,1.418108,0,0,0.000000,0,0
1,4,10 Cloverfield Lane,32676639.0,"3,391",Paramount Pictures,10 cloverfield lane,2016,333371.0,104.0,2016-03-10,...,3,0.866025,5.000000e-01,16.523561,2.251302,0,0,0.000000,0,0
2,9,102 Dalmatians,27376052.0,"2,704",Walt Disney Studios Motion Pictures,102 dalmatians,2000,10481.0,100.0,2000-10-07,...,10,-1.000000,-1.836970e-16,18.258162,2.237929,0,2,1.098612,0,0
3,13,12 Strong,21124610.0,"3,002",Warner Bros.,12 strong,2018,429351.0,130.0,2018-01-18,...,1,0.000000,1.000000e+00,17.370859,2.940177,0,0,0.000000,0,0
4,14,12 Years a Slave,1259943.0,19,Fox Searchlight Pictures,12 years a slave,2013,76203.0,134.0,2013-10-18,...,10,-1.000000,-1.836970e-16,16.811243,2.690897,0,0,0.000000,0,0


In [54]:
box_three.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2573 entries, 0 to 2572
Data columns (total 54 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Unnamed: 0                    2573 non-null   int64         
 1   Release                       2573 non-null   object        
 2   Seven_Day_Total               2573 non-null   float64       
 3   Theaters                      2573 non-null   object        
 4   Distributor                   2566 non-null   object        
 5   normalized_title              2573 non-null   object        
 6   year                          2573 non-null   int64         
 7   tmdb_id                       2573 non-null   float64       
 8   runtime                       2573 non-null   float64       
 9   release_date                  2573 non-null   datetime64[ns]
 10  budget                        2573 non-null   float64       
 11  genres                        

In [55]:
box_three['year']=pd.to_datetime(box['year'],errors='coerce')

In [56]:
box_three.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2573 entries, 0 to 2572
Data columns (total 54 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Unnamed: 0                    2573 non-null   int64         
 1   Release                       2573 non-null   object        
 2   Seven_Day_Total               2573 non-null   float64       
 3   Theaters                      2573 non-null   object        
 4   Distributor                   2566 non-null   object        
 5   normalized_title              2573 non-null   object        
 6   year                          2573 non-null   datetime64[ns]
 7   tmdb_id                       2573 non-null   float64       
 8   runtime                       2573 non-null   float64       
 9   release_date                  2573 non-null   datetime64[ns]
 10  budget                        2573 non-null   float64       
 11  genres                        

In [57]:
box_three['cast_top5_names'].isnull()

,cast_top5_names
0,False
1,False
2,False
3,False
4,False
...,...
2568,False
2569,False
2570,False
2571,False


In [60]:
box_three.drop(columns='Theaters')

,Unnamed: 0,Release,Seven_Day_Total,Distributor,normalized_title,year,tmdb_id,runtime,release_date,budget,...,release_month,month_sin,month_cos,budget_log,cast_top5_popularity_sum_log,is_franchise,franchise_size,franchise_size_log,installment_number,is_sequel
0,2,'83,2595812.0,Reliance Entertainment,83,1970-01-01 00:00:00.000002021,601337.0,160.0,2021-12-23,36786988.0,...,12,-5.000000e-01,8.660254e-01,17.420655,1.418108,0,0,0.000000,0,0
1,4,10 Cloverfield Lane,32676639.0,Paramount Pictures,10 cloverfield lane,1970-01-01 00:00:00.000002016,333371.0,104.0,2016-03-10,15000000.0,...,3,8.660254e-01,5.000000e-01,16.523561,2.251302,0,0,0.000000,0,0
2,9,102 Dalmatians,27376052.0,Walt Disney Studios Motion Pictures,102 dalmatians,1970-01-01 00:00:00.000002000,10481.0,100.0,2000-10-07,85000000.0,...,10,-1.000000e+00,-1.836970e-16,18.258162,2.237929,0,2,1.098612,0,0
3,13,12 Strong,21124610.0,Warner Bros.,12 strong,1970-01-01 00:00:00.000002018,429351.0,130.0,2018-01-18,35000000.0,...,1,0.000000e+00,1.000000e+00,17.370859,2.940177,0,0,0.000000,0,0
4,14,12 Years a Slave,1259943.0,Fox Searchlight Pictures,12 years a slave,1970-01-01 00:00:00.000002013,76203.0,134.0,2013-10-18,20000000.0,...,10,-1.000000e+00,-1.836970e-16,16.811243,2.690897,0,0,0.000000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2568,7312,Zombieland: Double Tap,35400273.0,Sony Pictures Releasing,zombieland double tap,1970-01-01 00:00:00.000002019,338967.0,99.0,2019-10-09,42000000.0,...,10,-1.000000e+00,-1.836970e-16,17.553180,3.050859,1,2,1.098612,1,0
2569,7313,Zookeeper,30052466.0,Sony Pictures Releasing,zookeeper,1970-01-01 00:00:00.000002011,38317.0,102.0,2011-07-06,80000000.0,...,7,1.224647e-16,-1.000000e+00,18.197537,2.466089,0,0,0.000000,0,0
2570,7314,Zoolander 2,18218011.0,Paramount Pictures,zoolander 2,1970-01-01 00:00:00.000002016,329833.0,100.0,2016-02-06,50000000.0,...,2,5.000000e-01,8.660254e-01,17.727534,2.668519,1,2,1.098612,1,0
2571,7316,Zootopia,92615126.0,Walt Disney Studios Motion Pictures,zootopia,1970-01-01 00:00:00.000002016,269149.0,109.0,2016-02-11,150000000.0,...,2,5.000000e-01,8.660254e-01,18.826146,2.811395,1,2,1.098612,1,0


##Processing Null Values

In [70]:
box_three['Distributor'].isnull().sum()

np.int64(7)

In [71]:
# 1) Fill missing
box_three["Distributor"] = box_three["Distributor"].fillna("Unknown")

# 2) Optional: normalize text
box_three["Distributor"] = (
    box_three["Distributor"]
    .str.strip()
    .str.lower()
)


In [72]:
box_three["production_companies"] = box_three["production_companies"].fillna("Unknown")


In [73]:
box_three["production_companies"] = (
    box_three["production_companies"]
    .fillna("unknown")
    .astype(str)
    .str.lower()
    .str.strip()
)

box_three["prodco_list"] = box_three["production_companies"].apply(
    lambda x: [] if x == "unknown"
    else [c.strip() for c in re.split(r"[|,;]", x) if c.strip()]
)

In [11]:
box.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2573 entries, 0 to 2572
Data columns (total 43 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Unnamed: 0                  2573 non-null   int64         
 1   Release                     2573 non-null   object        
 2   Seven_Day_Total             2573 non-null   float64       
 3   Theaters                    2573 non-null   object        
 4   Distributor                 2566 non-null   object        
 5   normalized_title            2573 non-null   object        
 6   year                        2573 non-null   int64         
 7   tmdb_id                     2573 non-null   float64       
 8   original_language           2573 non-null   object        
 9   runtime                     2573 non-null   float64       
 10  release_date                2573 non-null   datetime64[ns]
 11  budget                      2573 non-null   float64     

In [12]:
box[box['imdb_id'].isnull()]

,Unnamed: 0,Release,Seven_Day_Total,Theaters,Distributor,normalized_title,year,tmdb_id,original_language,runtime,...,num_posters,num_backdrops,imdb_id,total_marketing_assets,log_total_marketing_assets,director_weighted_rating,day_of_the_week,weekend_release,is_holiday,holiday_type
1937,5456,The Chosen: Last Supper,16059670.0,"2,234",-,the chosen last supper,2025,1380405.0,en,463.0,...,14.0,6.0,NaN,21.0,3.091042,NaN,3,0,0,none
1938,5457,The Chosen: Last Supper,16059670.0,"2,234",-,the chosen last supper,2025,1380405.0,en,463.0,...,5.0,1.0,NaN,7.0,2.079442,NaN,3,0,0,none


In [61]:
box_three.to_csv('/content/drive/MyDrive/Box office prediction dataset/master_vol_5.csv',index=False)

In [87]:
median_director_rating = box_three['director_weighted_rating'].median()
box_three['director_weighted_rating'] = box_three['director_weighted_rating'].fillna(median_director_rating)

##Creating theme flags for keywords

Instead of Multilabel encoding all the keywords or conducting a sentiment analysis, this approach was taken which essentially converts them into a binary flag for specific genres/themes for the movies.

In [63]:
# column name
COL = "keywords_count"

# sanity check
assert COL in box_three.columns, f"{COL} not found in dataframe"
assert (box_three[COL] >= 0).all(), "keywords_count contains negative values"

# log transform
box_three["keywords_count_log"] = np.log1p(box_three[COL].astype(float))

# quick verification
box_three[[COL, "keywords_count_log"]].head()


,keywords_count,keywords_count_log
0,6.0,1.945910
1,8.0,2.197225
2,11.0,2.484907
3,8.0,2.197225
4,14.0,2.708050


In [64]:
box_three["keywords_list"] = (
    box_three["keywords"]
    .fillna("")
    .str.lower()
    .str.split("|")   # adjust delimiter if needed
)


In [65]:
THEMES = {
    "kw_superhero": {'comic book', 'mask', 'mutant', 'origin story','superhero', 'supervillain', 'vigilante', 'villain'},
    "kw_animation": {"animation", "animated", "pixar", "dreamworks"},
    "kw_horror": {"horror", "slasher", "haunted", "zombie", "vampire"},
    "kw_romance": {"romance", "love", "romantic"},
    "kw_family": {"family", "kids", "children"},
    "kw_biopic": {"biography", "biopic", "based on true story"},
    "kw_fantasy_scifi": {"fantasy", "science fiction", "sci-fi", "space"},
    "kw_action_adventure": {"action", "adventure", "war"},
    'kw_crime_thriller': {'crime', 'detective', 'gangster', 'heist', 'mystery', 'noir', 'thriller'},
    'kw_comedy_drama': {'comedy', 'drama', 'dramedy', 'parody', 'satire'},
}


In [66]:
def has_theme(kws, vocab):
    return int(any(k in vocab for k in kws))

for col, vocab in THEMES.items():
    box_three[col] = box_three["keywords_list"].apply(
        lambda x: has_theme(x, vocab)
    ).astype("int8")


##Normalizing genres

In [67]:
# --- settings ---
GENRE_COL = "genres"   # change if your column name is different

# 1) Define canonical forms and simple synonym rules
CANON = {
    "action", "adventure", "animation", "comedy", "crime", "documentary",
    "drama", "family", "fantasy", "history", "horror", "music", "mystery",
    "romance", "science fiction", "tv movie", "thriller", "war", "western",
    "sport", "biography"
}

# simple regex-based synonym mapping: (pattern -> canonical)
SYNONYMS = [
    (r"\bsci[-\s]*fi\b|\bscience\s*f(i|iction)\b", "science fiction"),
    (r"\bro[-\s]*com\b|\brom[\s-]*comedy\b", "romance"),  # map romcoms to romance (optional)
    (r"\bdocu(mentary)?\b", "documentary"),
    (r"\bbiopic\b|\bbiography\b", "biography"),
    (r"\bfamily\s*(film|movie)?\b", "family"),
    (r"\btv\s*movie\b|\btelevision\s*movie\b", "tv movie"),
    (r"\bscifi\b", "science fiction"),
    (r"\bwarfare\b", "war"),
    (r"\bgangster\b", "crime"),
    (r"\bmusical\b", "music"),
    (r"\bsuperhero\b", "action"),     # optional: fold superhero under action
]

# 2) Normalizer: split, clean, map synonyms, dedupe, and sort
_SPLIT = re.compile(r"[|/,;]+")
_ALNUM_SPACE = re.compile(r"[^a-z0-9\s]+")

def _clean_token(t: str) -> str:
    t = t.lower().strip()
    t = _ALNUM_SPACE.sub("", t)     # keep alnum + space
    t = re.sub(r"\s+", " ", t).strip()
    return t

def _apply_synonyms(t: str) -> str:
    for pat, canon in SYNONYMS:
        if re.search(pat, t):
            return canon
    return t

def normalize_genres_cell(cell):
    if pd.isna(cell) or str(cell).strip() == "":
        return []
    # split on common delimiters
    raw = _SPLIT.split(str(cell).lower())
    norm = []
    for g in raw:
        g = _clean_token(g)
        if not g:
            continue
        g = _apply_synonyms(g)
        # keep only canonical or pass-through (optional: restrict to CANON)
        if g in CANON:
            norm.append(g)
        else:
            # optional: keep non-canonical tokens — comment out to drop
            norm.append(g)
    # de-duplicate while preserving order
    seen, out = set(), []
    for g in norm:
        if g not in seen:
            seen.add(g)
            out.append(g)
    # sort for stable representation (optional)
    out = sorted(out)
    return out

# 3) Apply to your DataFrame
box_three["genres_list"] = box_three[GENRE_COL].apply(normalize_genres_cell)

# 4) Create handy derived columns
box_three["genres_norm"] = box_three["genres_list"].apply(lambda xs: "|".join(xs) if xs else "")
box_three["num_genres"]  = box_three["genres_list"].apply(len).astype("int16")

In [68]:
box_three.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2573 entries, 0 to 2572
Data columns (total 69 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Unnamed: 0                    2573 non-null   int64         
 1   Release                       2573 non-null   object        
 2   Seven_Day_Total               2573 non-null   float64       
 3   Theaters                      2573 non-null   object        
 4   Distributor                   2566 non-null   object        
 5   normalized_title              2573 non-null   object        
 6   year                          2573 non-null   datetime64[ns]
 7   tmdb_id                       2573 non-null   float64       
 8   runtime                       2573 non-null   float64       
 9   release_date                  2573 non-null   datetime64[ns]
 10  budget                        2573 non-null   float64       
 11  genres                        

Documentary was not needed

In [69]:
# Remove any row where 'Documentary' is in the list of genres
# We use a lambda function to check inside every list safely
box_three = box_three[box_three['genres'].apply(lambda x: 'Documentary' not in x if isinstance(x, list) else True)]

# Verify it worked
print(f"Rows remaining: {len(box_three)}")

Rows remaining: 2573


##Quantifying Production backing
This feature block quantifies a film’s **production backing** and flags **major-studio involvement**, both strong proxies for financing capacity, distribution muscle, and marketing reach.

* `num_production_companies = len(prodco_list)` measures how many companies financed/produced the title; more producers often correlate with larger budgets and wider release plans, but with diminishing incremental impact.
* `has_major_studio` checks the producer list against a curated set of major studio aliases (Disney, Warner, Universal, Paramount, Sony/Columbia, 20th Century/Fox, Lionsgate, MGM) to capture the outsized effect of studio scale on opening performance.
* `num_prodcos_log = log1p(num_production_companies)` reduces right-skew and models diminishing returns so a film moving from 1→2 producers counts more than 9→10.
  Together these variables let models separate indie vs studio projects and gauge production scale without over-weighting extreme counts.


In [74]:
box_three["num_production_companies"] = box_three["prodco_list"].apply(len).astype("int16")


In [75]:
MAJOR_STUDIOS = {
    "disney", "walt disney",
    "warner bros", "warner brothers",
    "universal", "paramount",
    "sony", "columbia",
    "20th century", "fox",
    "lionsgate", "mgm"
}

def has_major(xs):
    return int(any(any(m in c for m in MAJOR_STUDIOS) for c in xs))

box_three["has_major_studio"] = box_three["prodco_list"].apply(has_major).astype("int8")


In [76]:
box_three["num_prodcos_log"] = np.log1p(box_three["num_production_companies"])


In [77]:
box_three[["production_companies", "prodco_list", "num_production_companies","has_major_studio"]].head(10)


,production_companies,prodco_list,num_production_companies,has_major_studio
0,"kabir khan films, vibri media, reliance entert...","[kabir khan films, vibri media, reliance enter...",6,0
1,bad robot,[bad robot],1,0
2,"walt disney pictures, cruella productions, kan...","[walt disney pictures, cruella productions, ka...",3,1
3,torridon films; alcon entertainment; black lab...,"[torridon films, alcon entertainment, black la...",5,0
4,"new regency pictures, plan b entertainment, ri...","[new regency pictures, plan b entertainment, r...",5,0
5,"fox searchlight pictures, pathé, everest enter...","[fox searchlight pictures, pathé, everest ente...",10,1
6,dimension films; im global; automatik entertai...,"[dimension films, im global, automatik enterta...",6,0
7,dreamworks pictures; reliance entertainment; n...,"[dreamworks pictures, reliance entertainment, ...",5,0
8,trident films; death row pictures; sumatra fil...,"[trident films, death row pictures, sumatra fi...",4,1
9,universal pictures; marc platt productions; ef...,"[universal pictures, marc platt productions, e...",9,1


In [78]:
box_three.loc[box_three["production_countries"].isna(), "production_countries"] = "us"


In [82]:
# split list strings
box_three["production_countries_list"] = box_three["production_countries"].fillna("").str.lower().str.split(r"[|,;]")

def normalize_countries(lst):
    clean = [c.strip() for c in lst if c.strip()]
    codes = coco.convert(names=clean, to="ISO2")
    # filter out failed matches ("not found")
    return [c for c in codes if c and c != "not found"]

box_three["production_countries_iso"] = box_three["production_countries_list"].apply(normalize_countries)

In [83]:
box_three[['production_countries_iso','production_countries_list']]

,production_countries_iso,production_countries_list
0,"[I, N]",[in]
1,"[U, S]",[us]
2,"[U, S]",[us]
3,"[U, S]",[united states of america]
4,"[GB, US]","[gb, us]"
...,...,...
2568,"[U, S]",[us]
2569,"[U, S]",[us]
2570,"[US, IT]","[us, it]"
2571,"[U, S]",[us]


In [84]:
def join_char_codes(xs):
    # if list looks like ['U','S'] (all 1-char tokens), join into a single code
    if xs and all(isinstance(x, str) and len(x) == 1 for x in xs):
        return [''.join(xs).upper()]
    return xs

box_three["production_countries_iso"] = box_three["production_countries_iso"].apply(join_char_codes)


In [85]:
box_three[['production_countries_iso','production_countries_list']]

,production_countries_iso,production_countries_list
0,[IN],[in]
1,[US],[us]
2,[US],[us]
3,[US],[united states of america]
4,"[GB, US]","[gb, us]"
...,...,...
2568,[US],[us]
2569,[US],[us]
2570,"[US, IT]","[us, it]"
2571,[US],[us]


In [86]:
box_three['is_US'] = box_three['production_countries_iso'].apply(lambda x: 1 if 'US' in x else 0)

# China flag
box_three['is_China'] = box_three['production_countries_iso'].apply(lambda x: 1 if 'CN' in x else 0)

# India flag
box_three['is_India'] = box_three['production_countries_iso'].apply(lambda x: 1 if 'IN' in x else 0)

# Count feature
box_three['num_countries'] = box_three['production_countries_iso'].apply(len)

##Normalizing MPAA ratings

In [79]:

COL_IN  = "certification"     # your raw column
COL_OUT = "mpaa_std"          # standardized MPAA column

# 1) Normalization + mapping (returns one of: G, PG, PG-13, R, NC-17, Unrated, or None)
def normalize_mpaa(s: str):
    if s is None or str(s).strip() == "":
        return None
    x = str(s).strip().lower()

    # exact/ordered patterns (most specific first)
    patterns = [
        (r"^nc[\s-]*17\b", "NC-17"),
        (r"^pg[\s-]*13\b", "PG-13"),
        (r"^tv[\s-]*ma\b", "R"),        # TV approximations
        (r"^tv[\s-]*14\b", "PG-13"),
        (r"^tv[\s-]*pg\b", "PG"),
        (r"^tv[\s-]*g\b", "G"),
        (r"^g(\b|$)|^general audiences$", "G"),
        (r"^pg(\b|$)|parental guidance", "PG"),
        (r"^r(\b|$)|^restricted$", "R"),
        (r"^(nr|not rated|unrated|ur)(\b|$)", "Unrated"),
        # common foreign → approximate MPAA (optional; adjust to taste)
        (r"^u(\b|$)", "G"),
        (r"^ua\b|^u/a\b", "PG-13"),
        (r"^12a\b|^12\b", "PG-13"),
        (r"^15\b", "R"),
        (r"^18\b", "NC-17"),
    ]
    for pat, lab in patterns:
        if re.search(pat, x):
            return lab
    return None  # keep as missing if unmapped

# 2) Apply normalization
box_three[COL_OUT] = box_three[COL_IN].apply(normalize_mpaa)

# 3) Missing flag BEFORE imputation
box_three["mpaa_missing"] = box_three[COL_OUT].isna().astype("int8")

# 4A) Simple, robust imputation: fill with "Unrated"
box_three[COL_OUT] = box_three[COL_OUT].fillna("Unrated")


In [80]:
box_three['mpaa_std']

,mpaa_std
0,PG-13
1,PG-13
2,Unrated
3,R
4,Unrated
...,...
2568,R
2569,PG
2570,PG-13
2571,PG


In [81]:
box_three['production_countries']

,production_countries
0,IN
1,US
2,US
3,United States of America
4,"GB, US"
...,...
2568,US
2569,US
2570,"US, IT"
2571,US


#Creating Feature List for model training

In [88]:
features=box_three.drop(columns=["Unnamed: 0",
"Release",
'year',
"normalized_title",
"collection_name",
'cast_top5_popularity_sum',
'num_trailers',
'num_teasers','num_featurettes','num_clips','num_posters','num_backdrops','total_marketing_assets','day_of_the_week','original_language_iso',
'keywords_list','genres_list','prodco_list','mpaa_missing',
'cast_top5_names',
'directors',
'writers',
'producers',
"tmdb_id",
'genres',
'production_companies',
'production_countries',
'collection_total_revenue',
'certification',
'collection_total_parts',
'collection_id',
'keywords',
'release_month','production_countries_iso','production_countries_list']
)

In [89]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2573 entries, 0 to 2572
Data columns (total 46 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Seven_Day_Total               2573 non-null   float64       
 1   Theaters                      2573 non-null   object        
 2   Distributor                   2573 non-null   object        
 3   runtime                       2573 non-null   float64       
 4   release_date                  2573 non-null   datetime64[ns]
 5   budget                        2573 non-null   float64       
 6   keywords_count                2573 non-null   float64       
 7   log_total_marketing_assets    2573 non-null   float64       
 8   director_weighted_rating      2573 non-null   float64       
 9   weekend_release               2573 non-null   int64         
 10  is_holiday                    2573 non-null   int64         
 11  holiday_type                  

In [90]:
box_three[box_three.isnull().any(axis=1)]

,Unnamed: 0,Release,Seven_Day_Total,Theaters,Distributor,normalized_title,year,tmdb_id,runtime,release_date,...,has_major_studio,num_prodcos_log,mpaa_std,mpaa_missing,production_countries_list,production_countries_iso,is_US,is_China,is_India,num_countries
0,2,'83,2595812.0,486,reliance entertainment,83,1970-01-01 00:00:00.000002021,601337.0,160.0,2021-12-23,...,0,1.945910,PG-13,0,[in],[IN],0,0,1,1
1,4,10 Cloverfield Lane,32676639.0,"3,391",paramount pictures,10 cloverfield lane,1970-01-01 00:00:00.000002016,333371.0,104.0,2016-03-10,...,0,0.693147,PG-13,0,[us],[US],1,0,0,1
2,9,102 Dalmatians,27376052.0,"2,704",walt disney studios motion pictures,102 dalmatians,1970-01-01 00:00:00.000002000,10481.0,100.0,2000-10-07,...,1,1.386294,Unrated,1,[us],[US],1,0,0,1
3,13,12 Strong,21124610.0,"3,002",warner bros.,12 strong,1970-01-01 00:00:00.000002018,429351.0,130.0,2018-01-18,...,0,1.791759,R,0,[united states of america],[US],1,0,0,1
4,14,12 Years a Slave,1259943.0,19,fox searchlight pictures,12 years a slave,1970-01-01 00:00:00.000002013,76203.0,134.0,2013-10-18,...,0,1.791759,Unrated,1,"[gb, us]","[GB, US]",1,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2566,7305,Zero Dark Thirty,868784.0,5,sony pictures releasing,zero dark thirty,1970-01-01 00:00:00.000002012,97630.0,157.0,2012-12-19,...,0,1.386294,R,0,[us],[US],1,0,0,1
2567,7308,Zola,2550468.0,"1,468",a24,zola,1970-01-01 00:00:00.000002021,539565.0,86.0,2021-06-30,...,0,1.386294,Unrated,1,[us],[US],1,0,0,1
2568,7312,Zombieland: Double Tap,35400273.0,"3,468",sony pictures releasing,zombieland double tap,1970-01-01 00:00:00.000002019,338967.0,99.0,2019-10-09,...,1,1.386294,R,0,[us],[US],1,0,0,1
2569,7313,Zookeeper,30052466.0,"3,482",sony pictures releasing,zookeeper,1970-01-01 00:00:00.000002011,38317.0,102.0,2011-07-06,...,1,1.791759,PG,0,[us],[US],1,0,0,1


In [91]:
features.head()

,Seven_Day_Total,Theaters,Distributor,runtime,release_date,budget,keywords_count,log_total_marketing_assets,director_weighted_rating,weekend_release,...,genres_norm,num_genres,num_production_companies,has_major_studio,num_prodcos_log,mpaa_std,is_US,is_China,is_India,num_countries
0,2595812.0,486,reliance entertainment,160.0,2021-12-23,36786988.0,6.0,3.610918,5.978574,0,...,drama|history,2,6,0,1.945910,PG-13,0,0,1,1
1,32676639.0,"3,391",paramount pictures,104.0,2016-03-10,15000000.0,8.0,4.804021,7.345529,0,...,drama|horror|science fiction|thriller,4,1,0,0.693147,PG-13,1,0,0,1
2,27376052.0,"2,704",walt disney studios motion pictures,100.0,2000-10-07,85000000.0,11.0,4.382027,6.980777,1,...,comedy|family,2,3,1,1.386294,Unrated,1,0,0,1
3,21124610.0,"3,002",warner bros.,130.0,2018-01-18,35000000.0,8.0,3.637586,6.345000,0,...,action|drama|history|war,4,5,0,1.791759,R,1,0,0,1
4,1259943.0,19,fox searchlight pictures,134.0,2013-10-18,20000000.0,14.0,5.370638,5.996000,1,...,drama|history,2,5,0,1.791759,Unrated,1,0,0,2


In [92]:
features['Theaters']

,Theaters
0,486
1,"3,391"
2,"2,704"
3,"3,002"
4,19
...,...
2568,"3,468"
2569,"3,482"
2570,"3,394"
2571,"3,827"


Fixing incosistencies in "Theaters" columns

In [93]:
features['Theaters'] = features['Theaters'].str.replace(',', '', regex=False).str.replace('-', '', regex=False).replace('', np.nan).astype(float)

In [94]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2573 entries, 0 to 2572
Data columns (total 46 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Seven_Day_Total               2573 non-null   float64       
 1   Theaters                      2563 non-null   float64       
 2   Distributor                   2573 non-null   object        
 3   runtime                       2573 non-null   float64       
 4   release_date                  2573 non-null   datetime64[ns]
 5   budget                        2573 non-null   float64       
 6   keywords_count                2573 non-null   float64       
 7   log_total_marketing_assets    2573 non-null   float64       
 8   director_weighted_rating      2573 non-null   float64       
 9   weekend_release               2573 non-null   int64         
 10  is_holiday                    2573 non-null   int64         
 11  holiday_type                  

In [95]:
features['Theaters']

,Theaters
0,486.0
1,3391.0
2,2704.0
3,3002.0
4,19.0
...,...
2568,3468.0
2569,3482.0
2570,3394.0
2571,3827.0


In [96]:
median_theaters = features['Theaters'].median()
features['Theaters'] = features['Theaters'].fillna(median_theaters)
print(f"NaN values in 'Theaters' after imputation: {features['Theaters'].isnull().sum()}")

NaN values in 'Theaters' after imputation: 0


In [97]:
features.isnull().sum()

,0
Seven_Day_Total,0
Theaters,0
Distributor,0
runtime,0
release_date,0
budget,0
keywords_count,0
log_total_marketing_assets,0
director_weighted_rating,0
weekend_release,0


Exporting features list.

In [98]:
features.to_csv('/content/drive/MyDrive/Box office prediction dataset/features.csv')